In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
file_path = '/content/drive/My Drive/Colab Notebooks/Research Techniques II/DF_cleaned_v1.csv'

In [3]:
import pandas as pd

df = pd.read_csv(file_path)
df.head()

,text,labels,word_count
0,"Başa düşmürəm, mən o vaxt onlardan necə gedə b...",0,13
1,Halal olsun admin heyyətinə. Tək tük səhifələr...,1,11
2,Dunya seyaheti etmek ucun limitsiz bilet ve pul,1,8
3,"O, bu məhəbbəti əməlləri ilə qazanmışdı”.",1,6
4,Maraqlıdır siğə söhbətini aralarında bölüşdürü...,0,13


In [4]:
print(df.columns)

Index(['text', 'labels', 'word_count'], dtype='object')


In [5]:
df['labels'].value_counts()

,count
labels,
0,13955
1,13894


In [17]:
corrections_df = pd.read_excel('/content/drive/My Drive/Colab Notebooks/Research Techniques II/word_correction_dict_2.xlsx')
word_correction_dict_2 = dict(zip(corrections_df['Column1'], corrections_df['Column2']))

In [18]:
import re

def correct_words_case_insensitive(text, correction_dict):
    def replace(match):
        original = match.group()
        lower = original.lower()
        if lower in correction_dict:
            corrected = correction_dict[lower]
            # Baş harf büyükse, düzeltmeyi de büyük harfli yap
            if original[0].isupper():
                return corrected.capitalize()
            else:
                return corrected
        return original

    pattern = re.compile(r'\b(' + '|'.join(re.escape(k) for k in correction_dict.keys()) + r')\b', flags=re.IGNORECASE)
    return pattern.sub(replace, str(text))

In [19]:
df['text'] = df['text'].apply(lambda x: correct_words_case_insensitive(x, word_correction_dict_2))

In [20]:
import re

def basic_cleaning(text):
    # 1. Web adreslerini kaldır
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # 2. En az iki kapanış parantezi olan ifadeleri kaldır
    text = re.sub(r'\){2,}', '', text)

    # 3. Çoklu boşlukları tek boşluğa indir
    text = re.sub(r'\s{2,}', ' ', text)

    return text.strip()

# Temizlemeyi uygula (sözlüksüz versiyon)
df_cleaned = df.copy()
df_cleaned['text'] = df_cleaned['text'].apply(basic_cleaning)

In [21]:
from sentence_transformers import SentenceTransformer

# LaBSE modelini yükle
model = SentenceTransformer('sentence-transformers/LaBSE')

# Temizlenmiş metinleri al
texts = df_cleaned['text'].tolist()

# Embedding çıkar (batch size isteğe göre artırılabilir)
embeddings = model.encode(texts, batch_size=64, show_progress_bar=True)

Batches:   0%|          | 0/436 [00:00<?, ?it/s]

In [22]:
import numpy as np

# NumPy array'e dönüştürmek istiyorsan:
embeddings = np.array(embeddings)

# Şekli kontrol et (satır sayısı, embedding boyutu)
print(embeddings.shape)

(27849, 768)


In [23]:
X_seq = np.expand_dims(embeddings, axis=1)

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_seq, df_cleaned['labels'].values, test_size=0.2, random_state=42)

In [25]:
from keras.models import Sequential
from keras.layers import Bidirectional, GRU, Dense, Input

model = Sequential()
model.add(Input(shape=(1, X_seq.shape[2])))  # (seq_len=1, embedding_dim=768)
model.add(Bidirectional(GRU(64)))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/5
697/697 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.7562 - loss: 0.5099 - val_accuracy: 0.7948 - val_loss: 0.4328
Epoch 2/5
697/697 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.7985 - loss: 0.4312 - val_accuracy: 0.7996 - val_loss: 0.4257
Epoch 3/5
697/697 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8050 - loss: 0.4150 - val_accuracy: 0.8011 - val_loss: 0.4241
Epoch 4/5
697/697 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8065 - loss: 0.4130 - val_accuracy: 0.7984 - val_loss: 0.4218
Epoch 5/5
697/697 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8124 - loss: 0.4033 - val_accuracy: 0.8000 - val_loss: 0.4199


In [27]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
              precision    recall  f1-score   support

           0       0.78      0.83      0.81      2786
           1       0.82      0.77      0.79      2784

    accuracy                           0.80      5570
   macro avg       0.80      0.80      0.80      5570
weighted avg       0.80      0.80      0.80      5570



In [29]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sentence_transformers import SentenceTransformer

test_texts = [# Negatif (0)
   "Reklamın reklamı da olar?",
    "Bunu işlətmək üçün diplom lazımdır sanki.",
    "Sadəcə quraşdırdım və peşman oldum.",
    "Gözləyirəm... hələ də gözləyirəm...",
    "Telefonum belə utanır bu tətbiqdən.",
    "Bu nə idi axı? Ciddi-ciddi soruşuram.",
    "2 dəqiqəlik iş üçün 10 səhifə icazə istəyir.",
    "Yenilədim və hər şey daha da pis oldu.",
    "Əsəb pozan dizayn, bir az da səs əlavə edin tam olsun.",
    "Bircə bu çatışmırdı, indi tamam oldu...",
    # Pozitiv (1)
    "Hər şey yerində, heç nə artıq deyil.",
    "Tətbiq mənim əhvalımı düzəldir, ciddi deyirəm.",
    "İlk dəfədir bu qədər rahat hiss edirəm.",
    "İçim rahatdır, nəhayət düzgün bir şey.",
    "Sadə, təmiz və işlək — daha nə olsun?",
    "Sanki beynimi oxuyub hazırlayıblar.",
    "Gecə-gündüz istifadə edirəm, bezmirəm.",
    "Tətbiq deyil, sənət əsəridir bu.",
    "İndi başa düşdüm niyə bu qədər yüklənib.",
    "Heç gözləmirdim, amma möhtəşəmdir."]


# 1. Test cümleleri ve etiketler
test_texts1 = [
    # Pozitiv
    "Bugün özümü çox yaxşı hiss edirəm.",
    "Hər şey planladığım kimi gedir.",
    "Sevdiyim musiqini dinləyəndə ruhum dincəlir.",
    "Səhər günəşi insanın içinə işıq saçır.",
    "Gözlənilmədən dostum zəng etdi, çox sevindim.",
    "Bu gün çox məhsuldar keçdi.",
    "Sadəcə sakitlik və sükunət, başqa heç nə istəmirəm.",
    "Küçədə uşaqların gülüşünü eşitmək çox gözəldir.",
    "Yaxşı insanlar hələ də var, sadəcə tapmaq lazımdır.",
    "Bu xəbəri eşidəndə çox xoşbəxt oldum.",
    "Axşam çayı ilə dincəlmək möhtəşəmdir.",
    "İnsan sevdiklərini yanında görəndə hər şey gözəldir.",
    "Təbiətə çıxmaq mənə çox enerji verir.",
    "Bir anlıq xoşbəxtlik bütün günü gözəl edə bilər.",
    "Gözlədiyim cavab müsbət oldu, şadam.",

    # Neqativ
    "Son vaxtlar heç nəyə həvəsim yoxdur.",
    "İnsanlar çox dəyişib, səmimiyyət azalıb.",
    "Dünən elə yorğun idim ki, heç nə edə bilmədim.",
    "Hər şey üst-üstə gəlir, nəfəs ala bilmirəm.",
    "Bu şəhərdə hər gün eyni darlıq var.",
    "Hər kəs öz işindədir, kimsə maraqlanmır.",
    "İçimdə qəribə bir boşluq var.",
    "Bu qədər səylə etdiyim iş dəyərləndirilmədi.",
    "Günlər keçir, amma heç nə dəyişmir.",
    "Səhərlər durmaq belə istəmirəm.",
    "Bu qədər səssizlik insanı dəli edir.",
    "Artıq heç kimə güvənmirəm.",
    "Yalnızlıq yavaş-yavaş adamı yeyir.",
    "Bəzən hamıdan uzaqlaşmaq istəyirəm.",
    "Həyat sanki yavaşca məni sıxır."
]
test_labels = [0]*10 + [1]*10

# 2. LaBSE modeli ile embedding
labse_model = SentenceTransformer("sentence-transformers/LaBSE")
X_manual = labse_model.encode(test_texts, convert_to_numpy=True)

# 3. Giriş boyutunu BiGRU modeline uygun hale getir (seq_len=1)
X_manual_seq = X_manual.reshape((X_manual.shape[0], 1, X_manual.shape[1]))

# 4. Tahmin yap (olasılık skorları)
pred_probs = model.predict(X_manual_seq).reshape(-1)

# 5. Eşik üstü ise 1, değilse 0 → tahmin etiketi
pred_labels = (pred_probs > 0.5).astype(int)

# 6. Sonuçları DataFrame ile göster
df_manual_test = pd.DataFrame({
    'text': test_texts,
    'true_label': test_labels,
    'predicted_label': pred_labels,
    'score': pred_probs  # burası 0–1 arası olasılık değeridir
})

# 6. Değerlendirme metrikleri
print(classification_report(df_manual_test['true_label'], df_manual_test['predicted_label']))

# 7. Görsel çıktı (Colab uyumlu)
from IPython.display import display
display(df_manual_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
              precision    recall  f1-score   support

           0       0.67      0.80      0.73        10
           1       0.75      0.60      0.67        10

    accuracy                           0.70        20
   macro avg       0.71      0.70      0.70        20
weighted avg       0.71      0.70      0.70        20



,text,true_label,predicted_label,score
0,Reklamın reklamı da olar?,0,0,0.213750
1,Bunu işlətmək üçün diplom lazımdır sanki.,0,0,0.138686
2,Sadəcə quraşdırdım və peşman oldum.,0,0,0.114854
3,Gözləyirəm... hələ də gözləyirəm...,0,1,0.927403
4,Telefonum belə utanır bu tətbiqdən.,0,0,0.041060
5,Bu nə idi axı? Ciddi-ciddi soruşuram.,0,0,0.022644
6,2 dəqiqəlik iş üçün 10 səhifə icazə istəyir.,0,1,0.763555
7,Yenilədim və hər şey daha da pis oldu.,0,0,0.083486
8,"Əsəb pozan dizayn, bir az da səs əlavə edin ta...",0,0,0.365952
9,"Bircə bu çatışmırdı, indi tamam oldu...",0,0,0.297835
